In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pickle
from sklearn.model_selection import train_test_split
from keras import metrics
from keras import regularizers
import math
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K

In [ ]:
pickle_file=open("D:/VIT/CAPSTONE/FINAL/DATASET/DEFECT_PARTS_LABELLED/PICKLE_FILES/x_train.pickle","rb")
x_train=pickle.load(pickle_file)
pickle_file.close()

pickle_file=open("D:/VIT/CAPSTONE/FINAL/DATASET/DEFECT_PARTS_LABELLED/PICKLE_FILES/y_train.pickle","rb")
y_train=pickle.load(pickle_file)
pickle_file.close()

y_train_new=[]
for y in y_train:
  y=y[...,np.newaxis]
  y_train_new.append(y)
y_train=np.array(y_train_new).astype(int)

x_train=x_train/255.0
y_train=((y_train)//2.0).astype(int)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.4,shuffle=True,random_state=42)

In [ ]:
def double_conv_block(x, n_filters):
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "LeakyReLU", kernel_regularizer=regularizers.l2(0.0001))(x)
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "LeakyReLU")(x)
   return x

In [ ]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.2)(p)
   return f, p

In [ ]:
def upsample_block(x, conv_features, n_filters):
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)

   x = layers.concatenate([x, conv_features])

   x = layers.Dropout(0.4)(x)

   x = double_conv_block(x, n_filters)

   return x

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

@keras.saving.register_keras_serializable()
class MaxPoolingWithArgmax2D(Layer):

    def __init__(
            self,
            pool_size=(2, 2),
            strides=(2, 2),
            padding='same',
            **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.padding = padding
        self.pool_size = pool_size
        self.strides = strides

    def call(self, inputs, **kwargs):
        padding = self.padding
        pool_size = self.pool_size
        strides = self.strides
        ksize = [1, *pool_size, 1]
        padding = padding.upper()
        strides = [1, *strides, 1]
        output, argmax = tf.nn.max_pool_with_argmax(
            inputs,
            ksize=ksize,
            strides=strides,
            padding=padding)

        argmax = K.cast(argmax, K.floatx())
        return [output, argmax]

    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [
            dim // ratio[idx]
            if dim is not None else None
            for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]

@keras.saving.register_keras_serializable()
class MaxUnpooling2D(Layer):
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = size

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        mask = K.cast(mask, 'int32')
        input_shape = tf.shape(updates, out_type='int32')

        if output_shape is None:
            output_shape = (
                input_shape[0],
                input_shape[1] * self.size[0],
                input_shape[2] * self.size[1],
                input_shape[3])

        ret = tf.scatter_nd(K.expand_dims(K.flatten(mask)),
                            K.flatten(updates),
                            [K.prod(output_shape)])

        input_shape = updates.shape
        out_shape = [-1,
                     input_shape[1] * self.size[0],
                     input_shape[2] * self.size[1],
                     input_shape[3]]
        return K.reshape(ret, out_shape)

    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        return (
            mask_shape[0],
            mask_shape[1] * self.size[0],
            mask_shape[2] * self.size[1],
            mask_shape[3]
        )

In [ ]:
def build_model():
  # inputs
  inputs = layers.Input(shape=(256,256,3))

  f1, p1 = downsample_block(inputs, 32)

  f2, p2 = downsample_block(p1, 64)

  pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size=(2, 2))(p2)

  unpool_1 = MaxUnpooling2D(size=(2, 2))([pool_1, mask_1])

  u8 = upsample_block(unpool_1, f2, 64)

  u9 = upsample_block(u8, f1, 32)

  outputs = layers.Conv2D(1, 1, padding="same", activation = "softmax")(u9)


    
  unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

  return unet_model

In [ ]:
model=build_model()

model.summary()

In [ ]:
model.compile(optimizer="Adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

checkpoint_filepath ="D:/VIT/CAPSTONE/FINAL/MODEL_BACKUP/model_1.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,monitor='accuracy',mode='max',save_best_only=True)

model_history = model.fit(x_train,y_train,epochs=100,batch_size=64,callbacks=[model_checkpoint_callback])